## FM Radio Tuning System (FINAL VERSION) ##

Created a FM radio tuning system similar to radios found in cars using an RTL-SDR, Arduino Mega, potentiometer for tuning, and buttons for preset FM stations and finding the strongest station. 

In [3]:
!pip install numpy matplotlib sounddevice
!pip install pyrtlsdr[lib]
# pyserial to interact with the Arduino Mega
!pip install pyserial

In [4]:
4+2 # Test if kernel is running

6

In [5]:
import numpy as np        # FFT
import sounddevice as sd  # Playing audio
from rtlsdr import RtlSdr # Interfacing with the RTL-SDR dongle hardware
from copy import copy     # Copy arrays

import serial             # Serial communication with Arduino
import time               # Timing control like inactivity and delays

In [6]:
# Closing any existing SDR instance from previous runs, preventing issues 
# like "device busy" errors when re-running the script
try: 
    sdr.close()
    print("Closed old SDR")
except NameError:
   print("No SDR instance found")

# Connecting arduino through serial communication over COM port
# Setting baud rate to 9600 and timeout to 1 second
try:
    arduino = serial.Serial('COM6', 9600, timeout=1)
    print("Connected to Arduino.")
except Exception as e:
    print(f"Failed to connect: {e}")

No SDR instance found
Connected to Arduino.


In [1]:
############################################ FUNCTIONS ################################################################
# convert potentiometer value to FM frequency range
def tuning(val):
    # 88MHz - 108MHz FM range
    # 0 - 1023 potentiometer range
    return val*((108e6-88e6)/1023) + 88e6 

# Scan FM band to find the strongest signal/station
def findStrongestStation():
    arduino.write(f"SCANNING:...\n".encode())

    # Create a new sdr object
    sdr = RtlSdr()

    # Define US FM band 
    fcmin = 88.1e6 # lowest freq in US FM band + 0.1 (the station starts at 88.1)
    fcmax = 108e6 # highest freq in US FM band
    fcstep = 200e3 # step by 200 kHz (standard FM spacing)

    # Define sample size and rate
    N = 256*256
    sdr.sample_rate = 2*256*256*16
    sdr.gain = 42.0

    # Bandpass filter cutoff
    fcutoff = 100000 # Cutoff frequency of filter 100kHz
    bpm = bandpassmask(N,sdr.sample_rate,fcutoff) # Create the bandpass mask

    # Track the strongest signal
    strongest = -np.inf
    bestfc = fcmin
    
    # loops through FM band
    for freq in np.arange(fcmin, fcmax, fcstep):
        sdr.center_freq = freq
        samples = sdr.read_samples(N) # Collect N samples...N must be multiple of 256
        spectrum = np.abs(np.fft.fftshift(np.fft.fft(samples))) # Strength of signal

        # Apply bandpass mask to filter the signal so that it only focuses on the FM band
        filteredspectrum = spectrum * bpm 

        # Strongest FM
        strength = np.max(filteredspectrum) 
        
        # if the strength is larger than the strongest strength previously
        if strength > strongest:
            strongest = strength
            bestfc = freq
            
    sdr.close()
    arduino.write(f"TUNING:{bestfc / 1e6:.2f}\n".encode())
    return bestfc

# from rtl_sdr_fm_audio_2025
# Create a bnadpass filter mask for a given sample count and cutoff frequency
def bandpassmask(N,fsps,fcutoff):
    fcutoff_n = fcutoff / fsps # fcutoff, normalized
    
    pbfw = round(2*fcutoff_n*N)
    sbw = int((N-pbfw)/2)

    res = np.concatenate((np.zeros(sbw),np.ones(pbfw),np.zeros(sbw)))
    return(res)

# Play audio from a specific FM frequency for a duration of Tmax seconds
def playAudio(fc, Tmax):
    arduino.write(f"LOADING:{fc / 1e6:.2f}\n".encode())
    sdr = RtlSdr() # Create a new sdr object
    fsps = 2*256*256*16 # ~2 Msps
    faudiosps =48000 # audio sampling frequency (for output)

    # Collecting samples
    N = round(fsps*Tmax) # N must be a multiple of 256
    sdr.sample_rate = fsps
    sdr.center_freq = fc
    sdr.gain = 42.0
    
    samples = sdr.read_samples(N)
    sdr.close()

    # Filtering Frequency
    spectrum = np.fft.fftshift(np.fft.fft(samples))
    fcutoff = 100000 # Cutoff frequency of filter 100kHz
    bpm = bandpassmask(N,fsps,fcutoff) # create the bandpass mask
    
    # Filter by applying frequency mask to spectrum
    filteredspectrum = spectrum * bpm

    # Convert masked spectrum back to time domain to get filtered signal 
    filteredsignal = np.fft.ifft(np.fft.fftshift(filteredspectrum)) # Good results

    #SQUELCH: suppress noise by zeroing weak parts of signal
    abssignal = np.abs(filteredsignal)
    meanabssignal = np.mean(abssignal)
    thetasquelched = np.angle(filteredsignal)
    squelchthresh = (meanabssignal/3.0)
    for i in range(N):
        if (abssignal[i]<squelchthresh):
            thetasquelched[i] = 0.0

    # ELIMINATE PHASE WRAP ERRORS
    # Compute derivative of theta signal
    # deriv (theta plus 0)
    derivthetap0 = np.convolve([1,-1],thetasquelched,'same')
    derivthetapp = np.convolve([1,-1],(thetasquelched+np.pi) % (2*np.pi),'same')
    # The 0, +pi comparison method
    # deriv (theta plus pi)
    derivtheta = np.zeros(len(derivthetap0))
    for i in range(len(derivthetap0)):
        if (abs(derivthetap0[i])<abs(derivthetapp[i])):
            derivtheta[i] = derivthetap0[i] 
        else:
            derivtheta[i] = derivthetapp[i]

    # Remove spike in phase derivative
    spikethresh = 2
    #***
    #cdtheta = copy(derivthetap0) # Cleaned derivative of theta
    cdtheta = copy(derivtheta) # Cleaned derivative of theta
    for i in range(1,len(derivtheta)-1):
        if (abs(derivtheta[i])>spikethresh):
            cdtheta[i] = (derivtheta[i-1]+derivtheta[i+1])/2.0

    # DOWNSAMPLING
    # down sampling factor
    dsf = round(fsps/faudiosps)

    #*** DownSampled Cleaned Derivative of Theta
    dscdtheta = cdtheta[::dsf]

    dscdtheta2 = copy(dscdtheta)
    for i in range(len(dscdtheta2)):
        dscdtheta2[i] = np.sum(cdtheta[i*dsf:(i+1)*dsf])/dsf

    dscdtheta_out = copy(dscdtheta) 
    
    # Play Audio
    arduino.write(f"FM:{fc / 1e6:.2f}\n".encode())
    # faudiosps defined at top (eg 48000)
    dt_audio = 1/faudiosps
    myaudio = dscdtheta_out
    sd.play(3*myaudio,faudiosps,blocking=True)

In [8]:
################################################# MAIN ###############################################
# Preset buttons for 4 FM stations
presetfc = {"PRESET1": 87.9e6, "PRESET2": 93.3e6, "PRESET3": 98.1e6, "PRESET4": 106.9e6}

# Keep track of tuning state
prevPotVal = None      # Previous potentiometer value
currfc = None          # Current frequency being tuned/played
inactiveTime = 3       # Time to wait before playing audio after tuning stops (in seconds)
potChange = 10         # Minimum change in potentiometer value to consider it to be different
prevTime = time.time() # Last time potentiometer was moved
potFlag = False        # Track if the potentiometer was moved
audioTime = 3          # Duration of audio playback in seconds

while True:
    # Read serial input from Arduino
    line = arduino.readline().decode().strip()

    # If the user is using the potentiometer to tune and let it set, then it plays the audio  
    if potFlag and currfc and (time.time()-prevTime) > inactiveTime:
        playAudio(currfc, audioTime)
        arduino.write(f"FM:{currfc / 1e6:.2f}\n".encode())
        currfc = None
        potFlag = False

    # If the line in the serial monitor is one of the presets then it plays the audio from the preset stations
    if line in presetfc:
        currfc = presetfc[line]
        potFlag = False
        prevPotVal = None
        prevTime = 0
        playAudio(currfc, audioTime)
        arduino.write(f"FM:{currfc / 1e6:.2f}\n".encode())

    # If user pressed the strongest station button, it will scan the different station and play the strongest FM station
    elif line == "STRONGEST":
        currfc = findStrongestStation()
        playAudio(currfc, audioTime)
        arduino.write(f"FM:{currfc / 1e6:.2f}\n".encode())
        currfc = None
        potFlag = False

    if line.startswith("POT:"):
        val = int(line.split(":")[1]) # ex: [POT]:[100]

        # Check if the potentiometer changed
        if prevPotVal is None or abs(val - prevPotVal) >= potChange:
            prevPotVal = val
            currfc = tuning(val)
            potFlag = True
            prevTime = time.time()

            # Real-time tuning on display
            arduino.write(f"TUNING:{currfc / 1e6:.2f}\n".encode())
    time.sleep(0.05) # Delay to prevent from overwhelming the loop
    

SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))